Speed Detection and Car tracking

In [2]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

try :
    from tracker import *
except ImportError :
    print('Import not done')

In [3]:
import ultralytics
import cv2 
import pandas as pd 
from ultralytics import YOLO


In [4]:
from tracker import*

In [5]:
model = YOLO('yolov8s.pt')

100%|██████████| 21.5M/21.5M [00:13<00:00, 1.62MB/s]


In [7]:
class_list = {'person', 'bicycle', 'car','motorcycle', 'airplane', 'bus',
              'train', 'trusk', 'traffic light'}

In [8]:
tracker = Tracker()
count = 0 

In [9]:
cap = cv2.VideoCapture('video/highway_mini.mp4')

In [11]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    frame = cv2.resize(frame,(1020,500))

    result = model.predict(frame)
    # print(result)
    a = result[0].boxes.data
    a = a.detach().cpu().numpy() # added this line
    px = pd.DataFrame(a).astype("float")
    print(px)


    cv2.imshow('frames', frame)
    if cv2.waitkey(0) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()